In [ ]:
import tensorflow as tf
import numpy as np

Make a basic layer.  Note that the base class is tf.Module, which allows you to keep track of variables and submodules.

In [ ]:
# Make some 
class Dense(tf.Module):
 def __init__(self, in_features, out_features, name=None):
   super().__init__(name=name)

   self.w = tf.Variable(
     tf.random.normal([in_features, out_features]), name='w')
   self.b = tf.Variable(tf.zeros([out_features]), name='b')

 def __call__(self, x):
   y = tf.matmul(x, self.w) + self.b
   return tf.nn.relu(y)

Now, make a model with two layers.  It also depends on tf.Module. 

You can (and should!) nest tf.Modules (or, by extension, Keras layers) inside each other.  

In [ ]:
class SequentialModule(tf.Module):
 def __init__(self, name=None):
   super().__init__(name=name)

   self.dense_1 = Dense(in_features=3, out_features=3)
   self.dense_2 = Dense(in_features=3, out_features=2)

 def __call__(self, x):
   r = self.dense_1(x)
   return self.dense_2(r)


# Instantiate the model.
myModel = SequentialModule()


Modules know about other modules, so you can look inside them.

In [ ]:
print("Variables:")
for variable in myModel.variables:
  print(variable)

print("\nModules:")
for module in myModel.submodules:
  print(module)

You can write all the variable values out into a file.

In [ ]:
# Save it
checkpoint = tf.train.Checkpoint(model=myModel)
checkpoint.write("apath.cpt")


You can look at what got written out:

In [ ]:
!ls apath*

The single shard are the variable values in a protobuf, and the index is metadata. 

## Training

Once you have a model, you need to train it.

Remember that training is:

* Build a model
* Choose an optimizer
* Do a forward pass
* Compare the output of the forward pass to the actual labels to generate a **loss*
* Using the gardient tape, use the loss to calculate your gradients
* Apply the gradient to the variables to improve your results (hopefully!)

Here is a tiny model with nonsense inputs and outputs that we can train for a single step to see it in action.

In [ ]:
model = SequentialModule()
x = tf.constant([[1., 2., 3.]])  # some input data
y = tf.constant([[2., 2.3]])  # some classification labels for this input data
optimizer = tf.keras.optimizers.SGD()  # Gradient descent optimizer

with tf.GradientTape() as tape:
  # Forward pass and calculate loss
  y_p = model(x)
  loss = tf.keras.losses.MSE(y_p, y)

# Calculate and apply gradients to minimize loss
grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))


In [ ]:
model.variables

## Models in Keras

The same model can work in Keras; just change the parent. Remember, Keras models are tf.Modules with extra features.

Keras also comes with a built-in dense layer, which will save you some time.

In [ ]:
class SequentialModel(tf.keras.Model):
 def __init__(self, name=None):
   super().__init__(name=name)

   self.dense_1 = tf.keras.layers.Dense(3, use_bias=False)
   self.dense_2 = tf.keras.layers.Dense(2, use_bias=False)

 def call(self, x):
   r = self.dense_1(x)
   return self.dense_2(r)


# Instantiate the model.
myModel = SequentialModel()
myModel.build(x.shape) # This allocates all the variables
myModel.summary()

Essentially the same model with Keras.  Keras has a lot more features, including helpful summaries, built-in bias and activation, and so on.  However, it's all built on top of `tf.Module`!

In real life, you might move your layer allocations into a build step for
Keras's convenience.

### Functional API

Keras also comes with a functional API for defining models, which can be elegant; it's optional.

In [ ]:
inputs = tf.keras.Input(shape=[3,])

x = tf.keras.layers.Dense(3)(inputs)
x = tf.keras.layers.Dense(2)(x)

my_functional_model = tf.keras.Model(inputs=inputs,
   outputs=x)

my_functional_model.build(x.shape)

my_functional_model.summary()